In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os
from pathlib import Path
import IPython.display as ipd

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
%matplotlib inline

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

import librosa
import librosa.display

In [15]:
# Importing required libraries 
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import pickle
import IPython.display as ipd

In [23]:
path = '/content/drive/MyDrive/voice/'
data_dir = '/content/drive/MyDrive/voice/all/'
test_dir = '/content/drive/MyDrive/voice/test/'

In [6]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'sample_submission.csv')

In [7]:
train.head()

,id,accent
0,common_voice_en_10047.wav,Africa
1,common_voice_en_10048.wav,Africa
2,common_voice_en_10049.wav,Africa
3,common_voice_en_10050.wav,Africa
4,common_voice_en_10051.wav,Africa


In [ ]:
from tqdm import tqdm
import librosa
# do feature extraction using librosa
def features_extract(file):
    # load the audio file
    audio,sample_rate = librosa.load(file_name,res_type='kaiser_fast')
    
    # extract the features
    feature = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=50)
    
    # feature scaling
    scaled_feature = np.mean(feature.T,axis=0)
    
    # return the scaled features
    return scaled_feature

# list containg all the features
extracted = []


# for each row in the csv
for index_num,row in tqdm(train.iterrows()):
    

    # get the file 
    file_name = os.path.join(os.path.abspath(data_dir),str(row['id']))

    # get file label
    final_class_labels = row['accent']
    
    # extract feature
    data= features_extract(file_name)
    
    # store it in a list
    extracted.append([data,final_class_labels])

In [10]:
extracted_df = pd.DataFrame(extracted,columns=['feature','class'])

In [11]:
extracted_df.to_csv(path+'extracted_df.csv', index=False)

In [12]:
x = np.array(extracted_df['feature'].tolist())
y = np.array(extracted_df['class'].tolist())

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

Y = to_categorical(le.fit_transform(y))

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size=0.2, random_state = 42)

# print the details
print("Number of training samples = ", x_train.shape[0])
print("Number of testing samples = ",x_test.shape[0])

Number of training samples =  20416
Number of testing samples =  5104


In [18]:
num_labels = Y.shape[1]
model = Sequential()

model.add(Dense(256, input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               13056     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               6

In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [22]:
num_epochs = 15
num_batch_size = 2

model.fit(
          x_train, 
          y_train, 
          batch_size=num_batch_size, 
          epochs=num_epochs,
          validation_data=(x_test, y_test),
         )

Epoch 1/15
160/160 [==============================] - 3s 11ms/step - loss: 1.3402 - accuracy: 0.3947 - val_loss: 1.3608 - val_accuracy: 0.3918
Epoch 2/15
160/160 [==============================] - 2s 10ms/step - loss: 1.3394 - accuracy: 0.3922 - val_loss: 1.3606 - val_accuracy: 0.3918
Epoch 3/15
160/160 [==============================] - 2s 10ms/step - loss: 1.3396 - accuracy: 0.3933 - val_loss: 1.3612 - val_accuracy: 0.3868
Epoch 4/15
160/160 [==============================] - 2s 9ms/step - loss: 1.3392 - accuracy: 0.3980 - val_loss: 1.3614 - val_accuracy: 0.3918
Epoch 5/15
160/160 [==============================] - 2s 10ms/step - loss: 1.3402 - accuracy: 0.3913 - val_loss: 1.3602 - val_accuracy: 0.3918
Epoch 6/15
160/160 [==============================] - 2s 10ms/step - loss: 1.3393 - accuracy: 0.3941 - val_loss: 1.3607 - val_accuracy: 0.3918
Epoch 7/15
160/160 [==============================] - 2s 10ms/step - loss: 1.3389 - accuracy: 0.3939 - val_loss: 1.3614 - val_accuracy: 0.3870


In [29]:
def get_id(data):
    return np.int(data.split("/")[-1].split(".")[-2])

test_ = pd.DataFrame(index = range(0, 6100), columns = ["path", "id"])
test_["path"] = glob(test_dir+"*.wav")
test_["id"] = test_["path"].apply(lambda x : get_id(x))

test_.head()

,path,id
0,/content/drive/MyDrive/voice/test/5593.wav,5593
1,/content/drive/MyDrive/voice/test/5594.wav,5594
2,/content/drive/MyDrive/voice/test/5592.wav,5592
3,/content/drive/MyDrive/voice/test/5577.wav,5577
4,/content/drive/MyDrive/voice/test/5585.wav,5585


In [ ]:
# list containg all the features
extracted = []


# for each row in the csv
for index_num,row in tqdm(test_.iterrows()):
    

    # get the file 
    file_name = os.path.join(os.path.abspath(test_dir),str(row['path']))

    # get file label
    #final_class_labels = row['accent']
    
    # extract feature
    data= features_extract(file_name)
    
    # store it in a list
    extracted.append([data])

In [33]:
extracted_df_test = pd.DataFrame(extracted,columns=['feature'])

In [34]:
extracted_df_test.to_csv(path+'extracted_df_test.csv', index=False)

In [35]:
x_test = np.array(extracted_df_test['feature'].tolist())

In [36]:
predicted_vector = np.argmax(model.predict(x_test), axis=-1)

In [37]:
predicted_class = le.inverse_transform(predicted_vector)

In [42]:
test['label'] = predicted_class